In [1]:
import pandas as pd
pd.options.display.max_columns = 50
import sqlite3
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from data_utils import (
    clean_transcripts,
    flesch,
    fog,
    smog,
    terms,
    uniqueterms, 
    ttr, 
    mtld, 
    hdd
)

con = sqlite3.connect("../data/abc.db")

In [2]:
df_abc = (pd.read_sql_query("SELECT * from abc_stories", con)
          .rename(columns={'publish_date': 'date'})
          .query('date==date')
          .assign(text=lambda df: df.text.str.strip())
          .query('text!=""')
          .assign(
              text=lambda df: df.text.apply(clean_transcripts),
              # Compute readability
              flesch_reading_ease=lambda df: df.text.apply(flesch),
              fog=lambda df: df.text.apply(fog), 
              smog=lambda df: df.text.apply(smog),  
              terms=lambda df: df.text.apply(terms),
              # Compute lexical richness
              uniqueterms=lambda df: df.text.apply(uniqueterms), 
              ttr=lambda df: df.text.apply(ttr),
              mtld=lambda df: df.text.apply(mtld),
              hdd=lambda df: df.text.apply(hdd),
         )
         )
        

df_abc.head(3)

,source,url,date,title,authors,text,flesch_reading_ease,fog,smog,terms,uniqueterms,ttr,mtld,hdd
0,ABC,https://abcnews.go.com/International/wireStory...,None,Russian missiles kill at least 21 in Ukraine's...,"[""Abc News""]",ukrainian authorities say russian missile atta...,39.679581,15.157563,16.798991,1015,436,0.437751,110.009189,0.859175
1,ABC,https://abcnews.go.com/Sports/wireStory/ap-sou...,None,"Big Ten votes to add USC, UCLA as members star...","[""Abc News""]",the big ten voted unanimously to add southern ...,46.859705,15.247665,16.556444,916,366,0.403974,75.500000,0.806306
2,ABC,https://abcnews.go.com/Entertainment/wireStory...,None,US newspapers continuing to die at rate of 2 e...,"[""Abc News""]",despite a growing recognition of the newspaper...,37.056366,NaN,18.478764,472,252,0.527197,92.555414,0.830429


In [3]:
(df_abc
 .drop('text', axis=1)
 .to_csv('../data/abc_2_textstat.csv.gz', compression="gzip")
)